In [2]:
# import torch
# from bilstm_crf import BiLSTMCRF
from torch import optim
from utils import *
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
import torch.nn as nn
from TorchCRF import CRF

# import evaluation metrics 
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

torch.manual_seed(1)

In [3]:
import torch.onnx

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')

In [4]:

class BiLSTMCRF(nn.Module):
    def __init__(self, vocab_size, word_embedding_dim, intent_embedding_dim, hidden_dim, output_dim, number_of_intents):
        
        super(BiLSTMCRF, self).__init__()
        # hyperparameters
        self.word_embedding_dim = word_embedding_dim
        self.inten_embedding_dim = intent_embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.output_dim = output_dim

        # model layers
        self.word_embedding = nn.Embedding(vocab_size, word_embedding_dim)
        self.intent_embedding = nn.Embedding(number_of_intents, intent_embedding_dim)
        self.lstm = nn.LSTM(word_embedding_dim + intent_embedding_dim, hidden_dim // 2, bidirectional=True, dropout=0.2)
        self.hidden_to_tag = nn.Linear(hidden_dim, output_dim)
        self.crf = CRF(output_dim)

    def __init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2), torch.randn(2, 1, self.hidden_dim // 2)) # initialize hidden state

    def __get_lstm_features(self, sentence, intent):
        self.hidden = self.__init_hidden()
        word_embeddings = self.word_embedding(sentence).view(len(sentence), 1, -1)
        # print("Word Embedding Shape:", word_embeddings.shape) # (len(sentence), batch_size, embedding_dimension)
        intent_embeddings = self.intent_embedding(intent).view(1, 1, -1).repeat(len(sentence), 1, 1)
        # print("Intent Embedding Shape:", intent_embeddings.shape) # (len(sentence), batch_size, embedding_dimension)
        embeddings = torch.cat((word_embeddings, intent_embeddings), dim=2) 
        # print("Embedding Shape:", embeddings.shape) # (len(sentence), batch_size, embedding_dimension * 2)
        lstm_out, self.hidden = self.lstm(embeddings, self.hidden)
        # print("LSTM Out Shape:", lstm_out.shape) # (len(sentence), batch_size, hidden_dimension)
        lstm_features = self.hidden_to_tag(lstm_out)
        # print("LSTM Features Shape:", lstm_features.shape) # (len(sentence), batch_size, output_dimension (number of target entities)) -> number of entities 3andena 30 
        return lstm_features

    def neg_log_likelihood(self, sentence, tags, intent, mask):
        # print("Intent Shape:", intent.shape)
        features = self.__get_lstm_features(sentence, intent) # emissions from the lstm layer
        # print("Features Shape:", features.shape)
        # print("NEG LIKE FEATURES:", features.shape)
        # print(features)
        # mask = mask.view(1, -1)
        # print("Mask Shape:", mask.shape)
        # repeated_mask = mask.repeat(features.shape[0], 1, 1).contiguous()
        # print(repeated_mask.shape)
        # print(repeated_mask)
        # features = features * mask
        # print("Intent:", intent[0])
        # print("Mask:", mask)
        # print("Features:", features)
        # print("Features Shape:", features.shape)
        # features = features * mask
        tags = tags.view(-1, 1)
        loss = -self.crf(features, tags)
        # print(loss)
        return loss

    def forward(self, sentence, intent, mask):
        lstm_features = self.__get_lstm_features(sentence, intent) # emissions from the lstm layer
        # print("LSTM Features", lstm_features)
        # print("Forward LSTM features:", lstm_features.shape)
        # repeated_mask = mask.repeat(lstm_features.shape[0], 1, 1).contiguous()
        # test_mask = torch.ones(len(sentence), 1, self.output_dim)
        # print("Repeated Mask Shape:", repeated_mask.shape)
        # print("Mask Shape:", mask.shape)
        # lstm_features = lstm_features * mask
        # print("Forward LSTM features:", lstm_features.shape)
        tag_sequence = self.crf.decode(lstm_features)
        # print("Tag Sequence:", tag_sequence)
        return tag_sequence
    

### **HELPER FUNCTIONS**

In [5]:
# constants
WORD_EMBEDDING_DIM = 50
INTENT_EMBEDIING_DIM = 50
HIDDEN_DIM = 64

In [6]:
def read_dataset():
    data = pd.read_csv('../ner_dataset/ner_dataset.csv', encoding='latin1')

    # remove white spaces from column names
    data.columns = data.columns.str.strip()

    print(data.columns)
    # Group by 'Sentence #' and aggregate
    grouped_data = data.groupby('Sentence #').agg({
        'Word': lambda x: ''.join(x),  # Join words into a single sentence
        'Tag': lambda x: list(x.str.strip()),       # Collect tags into a list
        'Intent': lambda x: list(x.str.strip().str.replace('_', ' '))     # Collect intents into a list
    }).reset_index()  # Reset index to make 'Sentence #' a regular column

    return data, grouped_data


def prepare_data(dataframe):
    dataset = []
    for _, row in dataframe.iterrows():
        sentence = row['Word'][1:]
        tags = row['Tag']
        intents = row['Intent']
        dataset.append((sentence, tags, intents[0]))

    return dataset

In [7]:
# Format data
data, goruped_data = read_dataset()
print(data["Intent"].unique())

goruped_data.head()

Index(['Sentence #', 'Word', 'Tag', 'Intent'], dtype='object')
[' variable declaration' ' function declaration' ' class declaration'
 ' assignment operation' ' array operation' ' bitwise operation'
 ' mathematical operation' ' membership operation' ' casting'
 ' io operation' ' assertion' ' libraries' ' file system' ' ide operation'
 ' comment' ' conditional operation' ' iterative operation']


,Sentence #,Word,Tag,Intent
0,0,make start time as double and initialize 0.00...,"[O, B-VAR, I-VAR, O, B-TYPE, O, O, B-VAL]","[variable declaration, variable declaration, v..."
1,1,declare min value as integer and value 131313,"[O, B-VAR, I-VAR, O, B-TYPE, O, O, B-VAL]","[variable declaration, variable declaration, v..."
2,2,define settings as boolean and value false,"[O, B-VAR, O, B-TYPE, O, O, B-VAL]","[variable declaration, variable declaration, v..."
3,3,define y as integer and assign to 12345,"[O, B-VAR, O, B-TYPE, O, O, O, B-VAL]","[variable declaration, variable declaration, v..."
4,4,initialize k as string and initialize it with...,"[O, B-VAR, O, B-TYPE, O, O, O, O, B-VAL, I-VAL]","[variable declaration, variable declaration, v..."


In [8]:
training_data = prepare_data(goruped_data)

print(training_data[0])

('make start time as double and initialize 0.000123', ['O', 'B-VAR', 'I-VAR', 'O', 'B-TYPE', 'O', 'O', 'B-VAL'], 'variable declaration')


### **PREPARE DATA**

In [9]:
# convert intent to index
with open('../ner_dataset/annotations/annotations.json', 'r') as f:
    data = json.load(f)

annotations = data['annotations']

intents = list(annotations.keys())

print(intents)
print(len(intents))

intent_to_index = {intent: index for index, intent in enumerate(intents)}

intent_to_index["<UNK>"] = len(intent_to_index)

['variable declaration', 'function declaration', 'class declaration', 'assignment operation', 'array operation', 'bitwise operation', 'mathematical operation', 'membership operation', 'casting', 'io operation', 'assertion', 'libraries', 'file system', 'ide operation', 'comment', 'conditional operation', 'iterative operation']
17


In [10]:
# convert data to indices
word_to_index = {}
tag_to_index = {}
for sentence, tags, intents in training_data:
    for word in sentence.split():
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)
            
    for tag in tags:
        if tag not in tag_to_index:
            tag_to_index[tag] = len(tag_to_index)

# add the 'UNK' token
word_to_index['<UNK>'] = len(word_to_index)

tag_to_index['<UNK>'] = len(tag_to_index)

tag_to_index

{'O': 0,
 'B-VAR': 1,
 'I-VAR': 2,
 'B-TYPE': 3,
 'B-VAL': 4,
 'I-VAL': 5,
 'B-FUNC': 6,
 'I-FUNC': 7,
 'B-PARAM': 8,
 'I-PARAM': 9,
 'B-CLASS_NAME': 10,
 'I-CLASS_NAME': 11,
 'B-LHS': 12,
 'I-LHS': 13,
 'B-RHS': 14,
 'I-RHS': 15,
 'B-OPERATION': 16,
 'B-ELEMENT': 17,
 'I-ARRAY': 18,
 'B-ARRAY': 19,
 'I-ELEMENT': 20,
 'I-OPERATION': 21,
 'B-OPERATOR': 22,
 'B-OPERAND': 23,
 'I-OPERAND': 24,
 'I-OPERATOR': 25,
 'B-CONDITION': 26,
 'B-COLLECTION': 27,
 'I-COLLECTION': 28,
 'I-CONDITION': 29,
 'B-CAST_TYPE': 30,
 'B-INPUT': 31,
 'B-OUTPUT': 32,
 'B-LIB_NAME': 33,
 'B-ACTION': 34,
 'I-ACTION': 35,
 'B-DIR': 36,
 'B-LINE': 37,
 'B-FILE': 38,
 'I-COMMENT': 39,
 'B-COMMENT': 40,
 'B-LOG': 41,
 'B-LOOP': 42,
 'B-STEP': 43,
 'B-END': 44,
 'B-START': 45,
 '<UNK>': 46}

In [11]:
vocab_size = len(word_to_index)

print("Vocabulary Size: ", vocab_size)

number_of_tags = len(tag_to_index)

print("Number of Tags: ", number_of_tags)

number_of_intents = len(intent_to_index)

print("Number of Intents: ", number_of_intents)

Vocabulary Size:  1006
Number of Tags:  47
Number of Intents:  18


In [12]:
# Assertions
i = 0
for sentence, tags, intents in training_data:
    if len(sentence.split()) != len(tags):
        print(f"Example {i}: Sentence Length: {len(sentence.split())}, Tags Length: {len(tags)}")
        print("Example:", training_data[i])
    i +=1

for sentence, tags, intents in training_data:
    assert len(sentence.split()) == len(tags)

In [13]:
with open('../ner_dataset/intent_to_tags.json', 'r') as f:
    intent_to_tags = json.load(f)

print(intent_to_tags)

all_tags = list(tag_to_index.keys())

{'variable declaration': ['O', 'VAR', 'VAL', 'TYPE'], 'function declaration': ['O', 'FUNC', 'PARAM', 'RET_TYPE'], 'class declaration': ['O', 'CLASS_NAME'], 'assignment operation': ['O', 'LHS', 'RHS'], 'conditional operation': ['O', 'LHS', 'RHS', 'CONDITION', 'LOG'], 'iterative operation': ['O', 'LOOP', 'START', 'END', 'LHS', 'RHS', 'CONDITION', 'STEP'], 'array operation': ['O', 'ARRAY', 'OPERATION', 'ELEMENT'], 'bitwise operation': ['O', 'OPERATOR', 'OPERAND'], 'mathematical operation': ['O', 'OPERAND', 'OPERATOR', 'VAR'], 'membership operation': ['O', 'ELEMENT', 'COLLECTION', 'CONDITION'], 'casting': ['O', 'VAR', 'CAST_TYPE'], 'io operation': ['VAR', 'INPUT', 'OUTPUT'], 'assertion': ['O', 'VAR', 'VAL', 'CONDITION'], 'libraries': ['O', 'LIB_NAME'], 'file system': ['O', 'FILE', 'ACTION', 'DIR'], 'ide operation': ['O', 'ACTION', 'TYPE', 'LINE', 'FILE'], 'comment': ['O', 'COMMENT']}


### **INTENTS HANDLING**

In [14]:
# here we want to filter out the tags that are not relevent to the given intent
def create_mask(intent, all_tags, intent_to_tags):
    intent_tags = intent_to_tags[intent]
    final_tags = []

    # print(intent_tags)


    # create BI tags for the intent
    for tag in intent_tags:
        # print(tag)
        if tag == 'O': 
            final_tags.append(tag)
            continue
        
        final_tags.append('B-' + tag)
        final_tags.append('I-' + tag)

    # print(final_tags)
        
    # mask = [tag in final_tags for tag in all_tags]
    # print(mask)
    return torch.tensor([tag in final_tags for tag in all_tags], dtype=torch.int32)

print(create_mask('comment', all_tags, intent_to_tags))

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       dtype=torch.int32)


### **MODEL DEFINITION**

In [15]:
model = BiLSTMCRF(vocab_size, word_embedding_dim=WORD_EMBEDDING_DIM, intent_embedding_dim=INTENT_EMBEDIING_DIM, hidden_dim=HIDDEN_DIM, output_dim=number_of_tags, number_of_intents=number_of_intents)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [25]:
intent_to_tags

{'variable declaration': ['O', 'VAR', 'VAL', 'TYPE'],
 'function declaration': ['O', 'FUNC', 'PARAM', 'RET_TYPE'],
 'class declaration': ['O', 'CLASS_NAME'],
 'assignment operation': ['O', 'LHS', 'RHS'],
 'conditional operation': ['O', 'LHS', 'RHS', 'CONDITION', 'LOG'],
 'iterative operation': ['O',
  'LOOP',
  'START',
  'END',
  'LHS',
  'RHS',
  'CONDITION',
  'STEP'],
 'array operation': ['O', 'ARRAY', 'OPERATION', 'ELEMENT'],
 'bitwise operation': ['O', 'OPERATOR', 'OPERAND'],
 'mathematical operation': ['O', 'OPERAND', 'OPERATOR', 'VAR'],
 'membership operation': ['O', 'ELEMENT', 'COLLECTION', 'CONDITION'],
 'casting': ['O', 'VAR', 'CAST_TYPE'],
 'io operation': ['VAR', 'INPUT', 'OUTPUT'],
 'assertion': ['O', 'VAR', 'VAL', 'CONDITION'],
 'libraries': ['O', 'LIB_NAME'],
 'file system': ['O', 'FILE', 'ACTION', 'DIR'],
 'ide operation': ['O', 'ACTION', 'TYPE', 'LINE', 'FILE'],
 'comment': ['O', 'COMMENT']}

In [26]:
final_tags = {}
for intent in intent_to_tags.keys():
    final_tags[intent] = []
    for tag in intent_to_tags[intent]:
        if tag == 'O': 
            final_tags[intent].append(tag)
            continue
        
        final_tags[intent].append('B-' + tag)
        final_tags[intent].append('I-' + tag)   

final_tags

{'variable declaration': ['O',
  'B-VAR',
  'I-VAR',
  'B-VAL',
  'I-VAL',
  'B-TYPE',
  'I-TYPE'],
 'function declaration': ['O',
  'B-FUNC',
  'I-FUNC',
  'B-PARAM',
  'I-PARAM',
  'B-RET_TYPE',
  'I-RET_TYPE'],
 'class declaration': ['O', 'B-CLASS_NAME', 'I-CLASS_NAME'],
 'assignment operation': ['O', 'B-LHS', 'I-LHS', 'B-RHS', 'I-RHS'],
 'conditional operation': ['O',
  'B-LHS',
  'I-LHS',
  'B-RHS',
  'I-RHS',
  'B-CONDITION',
  'I-CONDITION',
  'B-LOG',
  'I-LOG'],
 'iterative operation': ['O',
  'B-LOOP',
  'I-LOOP',
  'B-START',
  'I-START',
  'B-END',
  'I-END',
  'B-LHS',
  'I-LHS',
  'B-RHS',
  'I-RHS',
  'B-CONDITION',
  'I-CONDITION',
  'B-STEP',
  'I-STEP'],
 'array operation': ['O',
  'B-ARRAY',
  'I-ARRAY',
  'B-OPERATION',
  'I-OPERATION',
  'B-ELEMENT',
  'I-ELEMENT'],
 'bitwise operation': ['O',
  'B-OPERATOR',
  'I-OPERATOR',
  'B-OPERAND',
  'I-OPERAND'],
 'mathematical operation': ['O',
  'B-OPERAND',
  'I-OPERAND',
  'B-OPERATOR',
  'I-OPERATOR',
  'B-VAR',
  'I-

In [16]:
# Check predictions before training
precheck_tag = prepare_sequence(training_data[50][1], tag_to_index)
print(precheck_tag)

mask = create_mask(training_data[50][2], all_tags, intent_to_tags)

with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[50][0].split(), word_to_index)
    intent = training_data[50][2]
    # precheck_intent = prepare_sequence(intent[0], intent_to_index)
    precheck_intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)
    print(model(precheck_sent, precheck_intent, mask))

tensor([0, 0, 0, 1, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4])
[[26, 26, 43, 26, 11, 12, 2, 26, 0, 0, 0, 11, 14, 30, 12]]


c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\TorchCRF\__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ..\aten\src\ATen\native\TensorCompare.cpp:519.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


In [17]:
intent_to_index

{'variable declaration': 0,
 'function declaration': 1,
 'class declaration': 2,
 'assignment operation': 3,
 'array operation': 4,
 'bitwise operation': 5,
 'mathematical operation': 6,
 'membership operation': 7,
 'casting': 8,
 'io operation': 9,
 'assertion': 10,
 'libraries': 11,
 'file system': 12,
 'ide operation': 13,
 'comment': 14,
 'conditional operation': 15,
 'iterative operation': 16,
 '<UNK>': 17}

In [18]:
# training loop
def train(model, training_data, epochs=10):
    for epoch in tqdm(range(epochs)):
        total_loss = 0
        for sentence, tags, intent in training_data:
            # Step 1. Remember that Pytorch accumulates gradients.
            # We need to clear them out before each instance
            model.zero_grad()

            # Step 2. Get our inputs ready for the network, that is,
            # turn them into Tensors of word indices.
            intent_mask = create_mask(intent, all_tags, intent_to_tags)
            
            # print(intent)
            # print(intent_to_index[intent])
            # intent = prepare_sequence(intents[0], intent_to_index)
            # print(intent)
            sentence = prepare_sequence(sentence.split(), word_to_index)
            target_tags = torch.tensor([tag_to_index[t] for t in tags], dtype=torch.long)
            intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)


            # Step 3. Run our forward pass.
            loss = model.neg_log_likelihood(sentence, target_tags, intent, intent_mask)

            total_loss += loss.item()

            # Step 4. Compute the loss, gradients, and update the parameters by
            loss.backward()
            optimizer.step()
        
        print(f"Epoch: {epoch}, Loss: {total_loss / len(training_data)}")

train(model, training_data, epochs=50)

  2%|▏         | 1/50 [00:32<26:46, 32.79s/it]

Epoch: 0, Loss: 7.698023501867535


  4%|▍         | 2/50 [01:06<26:39, 33.32s/it]

Epoch: 1, Loss: 3.1732494366306


  6%|▌         | 3/50 [01:33<23:48, 30.39s/it]

...

Epoch: 47, Loss: 0.11076239531067596


 98%|█████████▊| 49/50 [20:45<00:17, 17.06s/it]

Epoch: 48, Loss: 0.1178958441197187


100%|██████████| 50/50 [21:02<00:00, 25.24s/it]

Epoch: 49, Loss: 0.11571337338151604


In [23]:
# Check predictions before training
precheck_tag = prepare_sequence(training_data[50][1], tag_to_index)
print(precheck_tag)

mask = create_mask(training_data[50][2], all_tags, intent_to_tags)

with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[50][0].split(), word_to_index)
    intent = training_data[50][2]
    # precheck_intent = prepare_sequence(intent[0], intent_to_index)
    precheck_intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)
    print(model(precheck_sent, precheck_intent, mask))

tensor([0, 0, 0, 1, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4])
[[0, 0, 0, 1, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4]]


In [20]:
def predict(model, test_data):
    predictions = [] 
    with torch.no_grad():
        for sentence, _, intent in test_data:
            precheck_sent = prepare_sequence(sentence.split(), word_to_index)
            # precheck_intent = prepare_sequence(intent, intent_to_index)
            precheck_intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)
            mask = create_mask(intent, all_tags, intent_to_tags)
            predictions.append(model(precheck_sent, precheck_intent, mask)[0])
    
    return predictions

In [37]:
# Check predictions before training
test_example_1 = ("create a variable called ay khara and set it to yasmine ghanem", ["O", "O", "O", "O", "B-VAR", "I-VAR", "O", "O", "O", "B-VAL", "I-VAL"], "variable declaration")
test_example_2 = ("cast x to an int", ["O", "B-VAR", "O", "O", "B-CAST_TYPE"], "casting")
test_example_3 = ("import the pandas library", ["O", "O", "B-LIB_NAME", "O"], "libraries")
test_example_4 = ('write a new comment multiply the emissions and transitions', ['O', 'O', 'O', 'B-COMMENT', 'I-COMMENT', 'I-COMMENT', 'I-COMMENT', 'I-COMMENT' ], 'comment')
test_example_5 = ('define a new class person', ['O', 'O', 'O', 'O', 'B-CLASS_NAME'], 'class declaration')
test_example_6 = ('create a new function named add that takes parameters a and b', ['O', 'O', 'O', 'O', 'O', 'B-FUNC', 'O', 'O', 'O', 'B-PARAM', 'O', 'B-PARAM'], 'function declaration')
test_example_7 = ('create a new function named add that takes parameters num1 and num2', ['O', 'O', 'O', 'O', 'O', 'B-FUNC', 'O', 'O', 'O', 'B-PARAM', 'O', 'B-PARAM'], 'function declaration')

test_data = [test_example_1, test_example_2, test_example_3, test_example_4, test_example_5, test_example_6, test_example_7]

predictions = predict(model, test_data)
labels = []
predicted_tags = []

for index, prediction in enumerate(predictions):
    print("Test Example:", index)
    mapped_tags = [all_tags[tag] for tag in prediction]
    predicted_tags.append(mapped_tags)
    labels.append(test_data[index][1])
    print("Actual tags:", test_data[index][1])
    print("Predicted tags:", mapped_tags)

Test Example: 0
Actual tags: ['O', 'O', 'O', 'O', 'B-VAR', 'I-VAR', 'O', 'O', 'O', 'B-VAL', 'I-VAL']
Predicted tags: ['O', 'O', 'O', 'O', 'B-VAR', 'I-VAR', 'O', 'O', 'O', 'O', 'B-VAL', 'I-VAL']
Test Example: 1
Actual tags: ['O', 'B-VAR', 'O', 'O', 'B-CAST_TYPE']
Predicted tags: ['O', 'B-VAR', 'O', 'O', 'B-CAST_TYPE']
Test Example: 2
Actual tags: ['O', 'O', 'B-LIB_NAME', 'O']
Predicted tags: ['O', 'O', 'B-LIB_NAME', 'O']
Test Example: 3
Actual tags: ['O', 'O', 'O', 'B-COMMENT', 'I-COMMENT', 'I-COMMENT', 'I-COMMENT', 'I-COMMENT']
Predicted tags: ['O', 'I-COMMENT', 'O', 'I-COMMENT', 'I-COMMENT', 'O', 'I-COMMENT', 'I-COMMENT', 'O']
Test Example: 4
Actual tags: ['O', 'O', 'O', 'O', 'B-CLASS_NAME']
Predicted tags: ['O', 'O', 'O', 'O', 'B-CLASS_NAME']
Test Example: 5
Actual tags: ['O', 'O', 'O', 'O', 'O', 'B-FUNC', 'O', 'O', 'O', 'B-PARAM', 'O', 'B-PARAM']
Predicted tags: ['O', 'O', 'O', 'O', 'O', 'B-FUNC', 'O', 'O', 'O', 'O', 'O', 'O']
Test Example: 6
Actual tags: ['O', 'O', 'O', 'O', 'O', '

In [22]:
# evaluation 
def evaluate(y_true, y_pred):
    # y_true = []
    # y_pred = []
    # for index, prediction in enumerate(predictions):
    #     y_true.extend(test_data[index][1])
    #     y_pred.extend([all_tags[tag] for tag in prediction])
    mlb = MultiLabelBinarizer()
    y_true_binarized = mlb.fit_transform(y_true)
    y_pred_binarized = mlb.transform(y_pred)
    
    f1 = f1_score(y_true_binarized, y_pred_binarized, average='weighted')
    precision = precision_score(y_true_binarized, y_pred_binarized, average='weighted')
    recall = recall_score(y_true_binarized, y_pred_binarized, average='weighted')
    accuracy = accuracy_score(y_true_binarized, y_pred_binarized)
    
    return f1, precision, recall, accuracy

f1, precision, recall, accuracy = evaluate(labels, predicted_tags)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)

F1 Score: 0.8888888888888888
Precision: 0.8888888888888888
Recall: 0.8888888888888888
Accuracy: 0.6666666666666666


c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
